In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

## Mars Exploration Program Scrape

In [2]:
# Import url and parse the html
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [3]:
# latest news title and paragraph text
news_title = soup.find('div', class_='content_title').text
p_text = soup.find('div', class_='image_and_description_container').text

## JPL Scrape

In [4]:
# Import url and parse the html
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [5]:
# splinter: find the image url for the current Featured Mars Image - `featured_image_url`.
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/melissahaynes/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [6]:
# open the url and scrape the featured image
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space'
browser.visit(url + '/index.html')
html = browser.html
soup = bs(html, 'html.parser')

# Scrape the featured image location
header = soup.find_all('div', class_='header')
for item in header: 
    featured_img = soup.find('img',class_='headerimage fade-in')['src']
    
# Get the full image url 
featured_img_url = url + '/' + featured_img
print(featured_img_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg


## Mars Facts Scrape

In [7]:
# Scrape the facts table
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)

# Convert to dataframe 
facts_df = pd.DataFrame(tables[0])

# Convert to HTML table string
facts_html = facts_df.to_html()

## Mars Hemispheres Scrape

In [8]:
# Scrape pictures of each hemisphere 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [64]:
# Navigate to item section 
links = soup.find_all('a', class_='itemLink product-item')

# Create empty list of links 
link_list = []

# Loop through link section and get image info links 
for link in links:
    img_link = link['href']
    
    # Add image info to list 
    link_list.append(img_link)

print(link_list)

['/search/map/Mars/Viking/cerberus_enhanced', '/search/map/Mars/Viking/schiaparelli_enhanced', '/search/map/Mars/Viking/syrtis_major_enhanced', '/search/map/Mars/Viking/valles_marineris_enhanced']


In [65]:
# Generate full links to image info pages 
base_url = 'https://astrogeology.usgs.gov'

# Empty list for full links 
full_links = []

# Loop through link list and generate full link
for link in link_list:
    full_link = base_url + link
    full_links.append(full_link)

print(full_links)

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']


In [126]:
# Empty lists for full-size image links and titles
img_links = []
titles = []

# Scrape each img info link 
for link in full_links:
    response = requests.get(link)
    soup = bs(response.text, 'html.parser')
    
    # Navigate to title 
    title = soup.find('h2', class_='title').text
    titles.append(title)
    
    # Navigate to image link location
    downloads = soup.find('div', class_='downloads')
    download_list = downloads.find_all('ul')
    
    # Loop through list and scrape the .tif link
    for download in download_list:
        
        # Navigate to the download links 
        anchors = download.find_all('a')
        
        # Only get the full-size image (Original)
        tif = anchors[1] 
        
        # Scrape the link
        link = tif['href']
        
        # Add to list
        img_links.append(link)

['https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif', 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif', 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif', 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif']


In [ ]:
hemispheres = []
for title in titles:
    